In [1]:
import re
import sqlalchemy
from sqlalchemy import exc
import pandas as pd 
import psycopg2
import time
import datetime

In [2]:
post_host = "192.168.11.192"
post_port = "5432"
post_user = "postgres"
post_password = "123456"
post_db_name = "wb_data"

def conn_sqlalchemy(user,password,host,port,db_name):
    """
    make a connection to postgresql use sqlalchemy toolkit

    """
    try:
        url = 'postgresql://%s:%s@%s:%s/%s' %(user,password,host,port,db_name)
        engine = sqlalchemy.create_engine(url)

        

    except (Exception, exc.DisconnectionError) as e:
        return 1

    return engine


engine = conn_sqlalchemy(post_user,post_password,post_host,post_port,post_db_name) #  连接数据库        

In [3]:
def tweet_show(engine):
    '''
    读取common_user_tweet数据库中的用户id，发表时间和微博内容三列
    '''
#     engine = conn_sqlalchemy(user,password,host,port,db_name)
    sql = 'select user_id,created_at,content from common_user_tweet limit 10000'
#     df = pd.read_sql_query(sql,engine)
    df = pd.read_sql(sql,engine)
#     df_kol = df.loc[:,['kol_unique_name','_id','brand_unique_name','nick_name','province','birthday','user_type','fan_num']]
#     df_kol = df.loc[:,['_id','nick_name','province','birthday','user_type','fan_num']]
#     df_kol = df_kol.loc[(df_kol['user_type'])==0] # 提取标签为kol的用户
#     df_kol = df_kol.loc[:,['_id','nick_name','province','birthday','fan_num']]
    
    return df

content_df = tweet_show(engine)

In [4]:
def content_clean(text):
    '''
    把微博中的表情、#号、位置信息和@内容清洗掉
    '''
#     text = '#开心#今天好开心[嘻嘻] 苏州大学 显示地图 王思媛的微博视频 转发微博 分享图片//@马思柳:'
#     print(text)
    try:
        re_text = u'\[.*?]|#| \S*? 显示地图|转发微博|分享图片| \S*?的微博视频 |//@.*?:|@\S*? |@\S*?$'
        text = re.sub(re_text, '', text)    
        pure_text = ''
        for letter in text:
            if letter.isalpha() or letter==' ':
                pure_text += letter

        text = ' '.join(word for word in pure_text.split() if len(word)>1)

        return text
    except:
        pass

In [5]:
content_df.head()

,user_id,created_at,content
0,5416945667,2021-07-01 22:27:00,转发微博
1,5416945667,2021-07-01 06:12:00,分享图片 太原 显示地图
2,5416945667,2021-07-01 06:03:00,祝你幸福
3,5416945667,2021-07-01 05:46:00,转发微博
4,5416945667,2021-07-01 05:46:00,转发微博


In [6]:
#增加一列保存清洗过的微博内容
content_df['content_cleaned'] = ''
for i in range(content_df.shape[0]):
    content_df.loc[i,'content_cleaned'] = content_clean(content_df.loc[i,'content'])

In [7]:
#确保微博发布时间在近三个月内
start_time = '2021-04-13 08:30:00'
content_df = content_df[content_df['created_at'] > start_time]

以上为微博内容读取和清洗的部分

以下为根据微博内容和关键词表对照的部分

In [9]:
#从抽奖关键词库中提取关键词
lottery_keywords = []
with open('word_stock/keywords/lottery.txt',encoding='utf-8') as f:
    while True:
        lines = f.readline()    #按行读取抽奖关键词
        if not lines:
            break
            pass
        lottery_keywords.append(lines[0:-1])    #保存关键词
        pass
f.close()

#增加"抽奖关键词数"列，统计出现抽奖关键词的个数
content_df['lottery_word_num'] = 0

error = 0
nonerror = 0

for i in range(content_df.shape[0]):
    try:
        for keyword in lottery_keywords:
            if keyword in str(content_df.loc[i,'content_cleaned']):
                content_df.loc[i,'lottery_word_num'] += 1
        nonerror += 1

    except:
        error += 1
        pass
    
    
print(error)
print(nonerror)

20
9960


In [ ]:
# common_tweet_user_lottery[common_tweet_user_lottery['lottery_word_num']!=0]

In [10]:
#从健身关键词库中提取关键词
fitness_keywords = []
with open('word_stock/keywords/fitness.txt',encoding='utf-8') as f:
    while True:
        lines = f.readline()    #按行读取抽奖关键词
        if not lines:
            break
            pass
        fitness_keywords.append(lines[0:-1])    #保存关键词
        pass
f.close()

nonerror = 0
error = 0

#增加"健身关键词数"列，统计出现健身关键词的个数
content_df['fitness_word_num'] = 0
for i in range(content_df.shape[0]):
    try:
        for keyword in fitness_keywords:
            if keyword in str(content_df.loc[i,'content_cleaned']):
                content_df.loc[i,'fitness_word_num'] += 1
        nonerror += 1
    except:
        error += 1
        pass

print(error)
print(nonerror)

20
9960


In [11]:
#以用户id为单位统计抽奖关键词出现的个数
user_interest = content_df.groupby(['user_id']).sum()

In [12]:
user_interest[user_interest['fitness_word_num'] != 0]

,lottery_word_num,fitness_word_num
user_id,,
1025509673,12,1
1175455614,255,11
1422919762,279,13
1674578333,19,1
1797261550,24,8
...,...,...
7386171414,287,102
7389317155,6,6
7559186572,93,3
